# 建筑物提取

In [ ]:
!pip install rasterio dotmap albumentations keras easydict

In [9]:
#!/usr/bin/env python3
# coding=utf-8
import os
import time
from iobjectspy import open_datasource
from iobjectspy.ml.vision import Inference

## 设置输入数据路径

In [10]:
curr_dir = ''
data_dir = os.path.join(curr_dir, '..','..','example_data')
inference_dir = os.path.join(data_dir, 'inference')

## 设置输出数据路径

In [11]:
out_dir = os.path.join(curr_dir, '..','..','out')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

## 设置模型路径路径

In [12]:
model_path = os.path.join(curr_dir, '..','..','model')
building_dom_model = os.path.join(model_path, 'bin_cls_building', 'bin_cls_building.sdm')

# 基于DOM数据集进行建筑物提取

In [13]:
try:
    ds = open_datasource(os.path.join(inference_dir, 'building.udb'))

    start_time = time.time()
    result = Inference(input_data=ds['building_infer'], model_path=building_dom_model,
                       out_data=os.path.join(out_dir, 'out_building.udb'),
                       out_dataset_name='dataset_dom', ).binary_classify_infer(offset=0, result_type='region')
    end_time = time.time()
    print('提取完成，共耗时{}s，结果数据保存在 {} 数据源下 {} 数据集中'.format(
        end_time - start_time, os.path.join(out_dir, 'out_building.udb'), result))

finally:
    ds.close()

NameError: name 'ds' is not defined

# 基于DOM影像文件进行建筑物提取

In [6]:
dom_path = os.path.join(inference_dir, 'building_infer.tif')

start_time = time.time()
result = Inference(input_data=dom_path, model_path=building_dom_model,
                   out_data=os.path.join(out_dir, 'out_building.udb'),
                   out_dataset_name='file_dom').binary_classify_infer(offset=0, result_type='region')

end_time = time.time()
print('提取完成，共耗时{}s，结果数据保存在 {} 数据源下 {} 数据集中'.format(
    end_time - start_time, os.path.join(out_dir, 'out_building.udb'), result))


RuntimeError: SuperMap_licensefile_7C_featrue_not_found

# 输入输出都为numpy数组的建筑物提取

In [7]:
"""
support any numpy size
:return:
"""

dom_path = os.path.join(inference_dir, 'building_infer.tif')
import rasterio
import numpy as np
from iobjectspy.ml.vision._inference_collector import BinaryClassificationWithNumpy
dom_array = np.transpose(rasterio.open(dom_path).read(), (1, 2, 0))
start_time = time.time()

# 加载模型是一个耗时操作,预测前初始化对象会自动加载,所有预测结束后,关闭模型
extraction = BinaryClassificationWithNumpy(
    model_path=building_dom_model)
for i in range(5):
    result_numpy = extraction.infer(image_data=dom_array, offset=0)
extraction.close_model()
print(result_numpy)
end_time = time.time()
print('提取完成，共耗时{}s'.format(
    end_time - start_time))

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]]
提取完成，共耗时22.05914330482483s


# 输入输出都为numpy数组的建筑物提取,且numpy数组为底层预测的最小tile尺寸

In [8]:
"""
only support the model default size
:return:
"""

dom_path = os.path.join(inference_dir, 'building_infer.tif')
import rasterio
import numpy as np
from iobjectspy.ml.vision._inference_collector import BinaryClassificationWithTile
dom_array = np.transpose(rasterio.open(dom_path).read(), (1, 2, 0))
dom_array = dom_array[:640, :640, :]
start_time = time.time()

# 加载模型是一个耗时操作,预测前初始化对象会自动加载,所有预测结束后,关闭模型
extraction = BinaryClassificationWithTile(
    model_path=building_dom_model)
result_numpy = extraction.infer_tile(image_data=dom_array)
extraction.close_model()
print(result_numpy)
end_time = time.time()
print('提取完成，共耗时{}s'.format(
    end_time - start_time))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
提取完成，共耗时3.6827621459960938s
